In [5]:
import geopandas as gpd
import pandas as pd

In [49]:
crashes = gpd.read_file('https://opendata.arcgis.com/datasets/70392a096a8e431381f1f692aaa06afd_24.geojson')

In [46]:
ancs = gpd.read_file('https://opendata.arcgis.com/datasets/fcfbf29074e549d8aff9b9c708179291_1.geojson')

In [50]:
crashes = gpd.sjoin(crashes, ancs, how="inner", op='within')

In [51]:
crashes['YEAR'] = crashes.apply(lambda x: x.REPORTDATE[:4], axis=1)

In [52]:
crashes.head()

,OBJECTID_left,CRIMEID,CCN,REPORTDATE,ROUTEID,MEASURE,OFFSET,STREETSEGID,ROADWAYSEGID,FROMDATE,...,UNKNOWNINJURIESPASSENGER,geometry,index_right,OBJECTID_right,ANC_ID,WEB_URL,NAME,Shape_Length,Shape_Area,YEAR
0,165433760,28540859,20136546,2020-09-24T21:59:59,0,0.00,14.69,NaN,NaN,2020-09-24T04:00:00,...,0,POINT Z (-76.95067 38.89309 0.00000),28,29,7F,http://anc.dc.gov/page/advisory-neighborhood-c...,ANC 7F,14062.386239,4.039022e+06,2020
3,165433763,28540867,20136578,2020-09-24T22:10:57,0,0.00,26.06,NaN,NaN,2020-09-24T04:00:00,...,0,POINT Z (-76.94919 38.89509 0.00000),28,29,7F,http://anc.dc.gov/page/advisory-neighborhood-c...,ANC 7F,14062.386239,4.039022e+06,2020
15,165433775,28514804,20123473,2020-08-29T20:18:31,0,0.00,0.02,NaN,NaN,2020-08-29T04:00:00,...,0,POINT Z (-76.94487 38.87910 0.00000),28,29,7F,http://anc.dc.gov/page/advisory-neighborhood-c...,ANC 7F,14062.386239,4.039022e+06,2020
34,165433606,25287571,15084076,2015-06-07T01:35:00,12029702,506.63,0.02,3571.0,4195.0,2015-06-06T04:00:00,...,0,POINT Z (-76.94888 38.89334 0.00000),28,29,7F,http://anc.dc.gov/page/advisory-neighborhood-c...,ANC 7F,14062.386239,4.039022e+06,2015
72,165433644,27384889,17130330,2017-07-30T11:10:25,13010401,385.10,4.28,14671.0,37902.0,2017-07-29T04:00:00,...,0,POINT Z (-76.96102 38.88856 0.00000),28,29,7F,http://anc.dc.gov/page/advisory-neighborhood-c...,ANC 7F,14062.386239,4.039022e+06,2017


In [53]:
#Number of crashes thus far in 2020 by ward
pd.DataFrame(crashes.groupby(['YEAR', 'NAME']).size()).loc['2020']

,0
NAME,
ANC 1A,344
ANC 1B,472
ANC 1C,154
ANC 1D,54
ANC 2A,294
ANC 2B,355
ANC 2C,424
ANC 2D,29
ANC 2E,186
